# Combining and merging data sets

The data contained in pandas objects can be combined in various ways:

* [pandas.merge](https://pandas.pydata.org/docs/reference/api/pandas.merge.html) joins rows in DataFrames based on one or more keys. This function is familiar from SQL or other relational databases as it implements database join operations.
* [pandas.concat](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) concatenates or stacks objects along an axis.
* The instance methods [pandas.DataFrame.combine_first](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.combine_first.html)  or [pandas.Series.combine_first](https://pandas.pydata.org/docs/reference/api/pandas.Series.combine_first.html) allow overlapping data to be joined together to fill in missing values in one object with values from another.
* With [pandas.merge_asof](https://pandas.pydata.org/docs/reference/api/pandas.merge_asof.html) you can perform time series-based window joins between DataFrame objects.

## Database-like DataFrame joins

Merge or join operations combine data sets by linking rows with one or more keys. These operations are particularly important in relational, SQL-based databases. The merge function in pandas is the main entry point for applying these algorithms to your data.

In [1]:
import pandas as pd

In [2]:
books = pd.DataFrame({'Language': ['en', 'en', 'de', 'fr', 'de', 'de', 'en'],
                      'Range': range(7)})

updates = pd.DataFrame({'Language': ['de', 'en', 'pt'],
                        'Range': range(3)})

In [3]:
books

,Language,Range
0,en,0
1,en,1
2,de,2
3,fr,3
4,de,4
5,de,5
6,en,6


In [4]:
updates

,Language,Range
0,de,0
1,en,1
2,pt,2


This is an example of a 1:n relationship; the data in `df1` has multiple rows labelled `de` and `en`, while `df2` has only one row for each value in the key column. When we call `merge` with these objects, we get:

In [5]:
pd.merge(books, updates)

,Language,Range
0,en,1


> **Note:**
> 
> I did not specify which column to merge over. If this information is not specified, `merge` uses the overlapping column names as keys. However, it is good practice to state this explicitly:

In [6]:
pd.merge(books, updates, on='Language')

,Language,Range_x,Range_y
0,en,0,1
1,en,1,1
2,en,6,1
3,de,2,0
4,de,4,0
5,de,5,0


If the column names are different in each object, you can specify them separately:

In [7]:
books = pd.DataFrame({'Language': ['en', 'en', 'de', 'fr', 'de', 'de', 'en'],
                      'Range': range(7)})

updates = pd.DataFrame({'Language': ['de', 'en', 'es'],
                        'Range': range(3)})

pd.merge(books, updates, left_on='Language', right_on='Language')

,Language,Range_x,Range_y
0,en,0,1
1,en,1,1
2,en,6,1
3,de,2,0
4,de,4,0
5,de,5,0


The values `fr` and `es` and the corresponding data are missing in the result. By default, `merge` performs an *inner join*; the keys in the result are the intersection or common set in both tables. Other possible options are *left join*, *right join* and *outer join*. Outer join takes the union of the keys and combines the effect of using *left join* and *right join*:

In [8]:
pd.merge(books, updates, how='outer')

,Language,Range
0,en,0
1,en,1
2,de,2
3,fr,3
4,de,4
5,de,5
6,en,6
7,de,0
8,es,2


Various *join* types with `how` argument

Option | Behaviour
:----- | :--------
`how='inner'` | uses only the key combinations observed in both tables
`how='left'` | uses all key combinations found in the left table
`how='right'` | uses all key combinations found in the right table
`how='outer'` | uses all key combinations observed in both tables together

n:n relationships form the [Cartesian product](https://en.wikipedia.org/wiki/Cartesian_product) of the matching keys, for example:

In [9]:
pd.merge(books, updates, on='Language', how='left')

,Language,Range_x,Range_y
0,en,0,1.0
1,en,1,1.0
2,de,2,0.0
3,fr,3,NaN
4,de,4,0.0
5,de,5,0.0
6,en,6,1.0


Since there were three `en`-lines in the left `DataFrame` and one in the right DataFrame, there are three `en`-lines in the result. The join method only affects the unique key values that appear in the result.

To join multiple keys, pass a list of column names:

In [10]:
books = pd.DataFrame({'Title': ['Jupyter Tutorial',
                                'Jupyter Tutorial',
                                'PyViz Tutorial'],
                      'Language': ['de', 'en', 'de'],
                      'Range': [1, 2, 3]})

updates = pd.DataFrame({'Title': ['Jupyter Tutorial',
                                  'PyViz Tutorial',
                                  'Python Basics',
                                  'Python Basics'],
                        'Language': ['de', 'de', 'de', 'en'],
                        'Range': [4, 5, 6, 7]})

pd.merge(books, updates, on=['Title', 'Language'], how='outer')

,Title,Language,Range_x,Range_y
0,Jupyter Tutorial,de,1.0,4.0
1,Jupyter Tutorial,en,2.0,NaN
2,PyViz Tutorial,de,3.0,5.0
3,Python Basics,de,NaN,6.0
4,Python Basics,en,NaN,7.0
